In [15]:
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from skimage.io import imread
from colorthief import ColorThief
import os
from tqdm.auto import tqdm
from PIL import ImageFile

In [16]:
# Allow loading of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Get colors from images

In [2]:
# get all files in ../../crawling/contents_merged/batches_with_images
files = os.listdir('../../crawling/contents_merged/batches_with_images')
print(len(files))

111


In [6]:
def extract_palette(image_path, num_colors):
    color_thief = ColorThief(image_path)
    palette = color_thief.get_palette(color_count=num_colors)
    return palette

def get_image_colors(image_path):
    num_colors = 5
    colors = extract_palette(image_path, num_colors)

    # convert colors to hex
    colors = [f'#{color[0]:02x}{color[1]:02x}{color[2]:02x}' for color in colors]
    return colors

In [22]:
# loop over files and load them as a df
start_file = 109
batch_number = start_file
for file in tqdm(files[start_file:], total=len(files[start_file:]), leave=False):
    df = pd.read_csv(f'../../crawling/contents_merged/batches_with_images/{file}')
    
    # loop over rows and extract color palette (use tqdm)
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"Processing Batch {batch_number}", leave=False):
        try:
            if str(row['Image_Path']) != "nan":
                colors = get_image_colors("../../crawling" + row['Image_Path'][1:])
                df.loc[index, 'Color_Palette'] = str(colors)
            else:
                df.loc[index, 'Color_Palette'] = None
        except Exception as e:
            print(f'Error at index {index} with path: {row["Image_Path"]} and URL: {row["Image_URL"]}')
            raise e

    batch_number += 1

    # save df to csv to ../../crawling/contents_merged/batches_with_image_colors/
    df.to_csv(f'../../crawling/contents_merged/batches_with_image_colors/{file}', index=False)

  0%|          | 0/2 [00:00<?, ?it/s]

Processing Batch 109:   0%|          | 0/10000 [00:00<?, ?it/s]

Processing Batch 110:   0%|          | 0/7474 [00:00<?, ?it/s]

In [20]:
# test_df =pd.read_csv(f'../../crawling/contents_merged/batches_with_images/batch_109.csv')

# # change the url to be /media/catalogue/Vietnam/Postage-stamps/EEX-s.jpg

# test_df.loc[test_df['Image_URL'] == '/media/catalogue/Vietnam/Postage-stamps/EEX-i.jpg', 'Image_URL'] = '/media/catalogue/Vietnam/Postage-stamps/EEX-s.jpg'

# # save df to csv
# test_df.to_csv(f'../../crawling/contents_merged/batches_with_images/batch_109.csv', index=False)

# all errors:
Error at index 8985 with path: ./images_all_2/Burkina Faso/1995/89458/NE-i.jpg and URL: /media/catalogue/Burkina-Faso/Postage-stamps/NE-i.jpg

# merge all batches

In [23]:
# merge all batches which are in folder ../../crawling/contents_merged/batches_with_image_colors/

df = pd.concat([pd.read_csv(f'../../crawling/contents_merged/batches_with_image_colors/{file}') for file in os.listdir('../../crawling/contents_merged/batches_with_image_colors')], ignore_index=True)

In [24]:
# get amount of rows
df.shape[0]

1107474

In [29]:
# save df to all_contents_updated_4.csv
df.to_csv('../../crawling/contents_merged/all_contents_updated_4.csv', index=False)

In [27]:
# load all_contents_updated_3.csv
df_old = pd.read_csv('../../crawling/contents_merged/all_contents_updated_3.csv')

C:\Users\Robin\AppData\Local\Temp\ipykernel_2712\3720273778.py:2: DtypeWarning: Columns (12,13,14,15,16,21,27,28,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df_old = pd.read_csv('../../crawling/contents_merged/all_contents_updated_3.csv')


In [28]:
df_old.shape[0]

1107474